In [ ]:
###################################################################################################
#
# redshift_stacking_plots.py 	    (c) Ian Williams, Adnan Khan, Matt McQuinn
#     				    	        ianw89@live.com
#
#
#  This file generates any plots that needs uses the stacked runs (out to z=1).
#  This produces Figure 2 and 3 of Khan, Williams, McQuinn 2022
#
###################################################################################################

from cgmbrush.cgmbrush import *
from cgmbrush.plots.plots import *
import matplotlib.pyplot as plt
import numpy as np

curve_thickness = 2

plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=XBIG_SIZE)     # fontsize of the axes title
plt.rc('legend', fontsize=BIGGER_SIZE)    # legend fontsize

seed = '50g89Gfh03f4Gh0r38h2TfM08' # a consistent random seed to use 

In [ ]:
# Load / Generate the Stacked Fields
provider = BolshoiProvider()
resolution = 8
grid_size = resolution * 1024
load_data = True
load_stacked = True
data_date = '2022-04-04'
redshifts = RS_array_gen(1,provider.Lbox)

configs = [
    Configuration(SphericalTophatProfile(), 1, provider=provider, resolution=resolution),
    Configuration(SphericalTophatProfile(), 2, provider=provider, resolution=resolution),
    Configuration(NFWProfile(), 1, provider=provider, resolution=resolution),
    Configuration(FireProfile(), 1, provider=provider, resolution=resolution),
    Configuration(PrecipitationProfile(), 1, provider=provider, resolution=resolution)
]

for c in configs:
    c.RS_array = redshifts
    c.seed = seed
    c.datestamp = data_date 
    
STH_256_stacked = configs[0].get_stacked_final_field()
STH_2_256_stacked = configs[1].get_stacked_final_field()
NFW_256_stacked = configs[2].get_stacked_final_field()
fire_256_stacked = configs[3].get_stacked_final_field()
P_256_stacked = configs[4].get_stacked_final_field()

In [ ]:
# Sum up the DM along the stack
# This works out to be approximately correct for these 250 h^-1 Mpc boxes
STH_DM_to_0_1 = sum(STH_256_stacked[0:1,:,:])
STH_DM_to_0_5 = sum(STH_256_stacked[0:5,:,:])
STH_DM_to_1 = sum(STH_256_stacked)

NFW_DM_to_0_1 = sum(NFW_256_stacked[0:1,:,:])
NFW_DM_to_0_5 = sum(NFW_256_stacked[0:5,:,:])
NFW_DM_to_1 = sum(NFW_256_stacked)

fire_DM_to_0_1 = sum(fire_256_stacked[0:1,:,:])
fire_DM_to_0_5 = sum(fire_256_stacked[0:5,:,:])
fire_DM_to_1 = sum(fire_256_stacked)

STH_2_DM_to_0_1 = sum(STH_2_256_stacked[0:1,:,:])
STH_2_DM_to_0_5 = sum(STH_2_256_stacked[0:5,:,:])
STH_2_DM_to_1 = sum(STH_2_256_stacked)

P_DM_to_0_1 = sum(P_256_stacked[0:1,:,:])
P_DM_to_0_5 = sum(P_256_stacked[0:5,:,:])
P_DM_to_1 = sum(P_256_stacked)

# Creating histograms for desired redshift

STH_RS_tenth= create_histograms(STH_256_stacked[0:1,:,:], grid_size)
STH_RS_half = create_histograms(STH_256_stacked[0:5,:,:], grid_size)
STH_RS_one = create_histograms(STH_256_stacked, grid_size)

NFW_RS_tenth= create_histograms(NFW_256_stacked[0:1,:,:], grid_size)
NFW_RS_half = create_histograms(NFW_256_stacked[0:5,:,:], grid_size)
NFW_RS_one = create_histograms(NFW_256_stacked, grid_size)

fire_RS_tenth= create_histograms(fire_256_stacked[0:1,:,:], grid_size)
fire_RS_half = create_histograms(fire_256_stacked[0:5,:,:], grid_size)
fire_RS_one = create_histograms(fire_256_stacked, grid_size)

STH_2_1 = create_histograms(STH_2_256_stacked[0:1,:,:], grid_size)
STH_2_5 = create_histograms(STH_2_256_stacked[0:5,:,:], grid_size)
STH_2_10 = create_histograms(STH_2_256_stacked, grid_size)

P_tenth = create_histograms(P_256_stacked[0:1,:,:], grid_size)
P_half = create_histograms(P_256_stacked[0:5,:,:], grid_size)
P_one = create_histograms(P_256_stacked, grid_size)


In [ ]:
def variance(profile, to_0_1, to_0_5, to_1):
    print('Mean of %s' % profile,'RS 0.1 = ',np.round(to_0_1.mean(),2) ,' RS 0.5 = ', np.round(to_0_5.mean(),2) ,' RS 1 = ',np.round(to_1.mean(),2) )
    print('Std Dev of %s' % profile,'RS 0.1 = ',np.round(np.sqrt(to_0_1.var()),2) ,' RS 0.5 = ', np.round(np.sqrt(to_0_5.var()),2) ,' RS 1 = ',np.round(np.sqrt(to_1.var()),2) )

variance('Tophat', STH_DM_to_0_1, STH_DM_to_0_5, STH_DM_to_1)
variance('Tophat 2 Rvir', STH_2_DM_to_0_1, STH_2_DM_to_0_5, STH_2_DM_to_1)
variance('NFW', NFW_DM_to_0_1, NFW_DM_to_0_5, NFW_DM_to_1)
variance('Fire', fire_DM_to_0_1, fire_DM_to_0_5, fire_DM_to_1)
variance('Precipitation', P_DM_to_0_1, P_DM_to_0_5, P_DM_to_1)

# This is used in DM vs R plots


In [ ]:
# Makes Figure 3, the 3 histograms for out to 3 redshifts

def make_histograms():

    fig, axs = plt.subplots(1, 3, sharex='col', sharey='row',
                            gridspec_kw={'hspace': 0, 'wspace': 0},figsize=(20,20*15 / 45))

    curve_lw = curve_thickness

    axs[0].plot(STH_RS_tenth[0],STH_RS_tenth[0]*STH_RS_tenth[1],'-',label='3D Tophat',lw=curve_lw,color='red')
    axs[0].plot(STH_2_1[0],STH_2_1[0]*STH_2_1[1],'-',label='3D Tophat 2$R_{vir}$',lw=curve_lw,color='orange')
    axs[0].plot(NFW_RS_tenth[0],NFW_RS_tenth[0]*NFW_RS_tenth[1],'-',label='NFW',lw=curve_lw,color='blue')
    axs[0].plot(fire_RS_tenth[0],fire_RS_tenth[0]*fire_RS_tenth[1],'-',label='Fire',lw=curve_lw,color='green')
    axs[0].plot(P_tenth[0],P_tenth[0]*P_tenth[1],'-',label='Precipitation',lw=curve_lw,color='c')

    axs[1].plot(STH_RS_half[0],STH_RS_half[0]*STH_RS_half[1],'-',label='3D Tophat',lw=curve_lw,color='red')
    axs[1].plot(STH_2_5[0],STH_2_5[0]*STH_2_5[1],'-',label='3D Tophat 2Rvir',lw=curve_lw,color='orange')
    axs[1].plot(NFW_RS_half[0],NFW_RS_half[0]*NFW_RS_half[1],'-',label='NFW',lw=curve_lw,color='blue')
    axs[1].plot(fire_RS_half[0],fire_RS_half[0]*fire_RS_half[1],'-',label='Fire',lw=curve_lw,color='green')
    axs[1].plot(P_half[0],P_half[0]*P_half[1],'-',label='Fire',lw=curve_lw,color='c')

    axs[2].plot(STH_RS_one[0],STH_RS_one[0]*STH_RS_one[1],'-',label='3D Tophat',lw=curve_lw,color='red')
    axs[2].plot(STH_2_10[0],STH_2_10[0]*STH_2_10[1],'-',label='3D Tophat 2Rvir',lw=curve_lw,color='orange')
    axs[2].plot(NFW_RS_one[0],NFW_RS_one[0]*NFW_RS_one[1],'-',label='NFW',lw=curve_lw,color='blue')
    axs[2].plot(fire_RS_one[0],fire_RS_one[0]*fire_RS_one[1],'-',label='Fire',lw=curve_lw,color='green')
    axs[2].plot(P_one[0],P_one[0]*P_one[1],'-',label='Fire',lw=curve_lw,color='c')

    # Axis range
    axs[0].set_xlim(xmin=0,xmax=199)
    axs[1].set_xlim(xmin=0,xmax=1499)
    axs[2].set_xlim(xmin=0,xmax=2500)

    # Subplot titles
    axs[0].set_title('Redshift = 0.1') 
    axs[1].set_title('Redshift = 0.5')
    axs[2].set_title('Redshift = 1.0')

    # Mean DM: set for the given redshift
    mean=axs[0].axvline(np.mean(STH_DM_to_0_1), color='k', linestyle='-', linewidth=0.5)
    axs[1].axvline(np.mean(STH_DM_to_0_5), color='k', linestyle='-', linewidth=0.5)
    axs[2].axvline(np.mean(STH_DM_to_1), color='k', linestyle='-', linewidth=0.5)
    # mean.set_label('Mean DM')

    # Axis labels
    axs[1].set_xlabel('DM [pc $cm^{-3}$]', fontsize=XBIG_SIZE)
    axs[0].set_ylabel(r'$ P(DM) \times DM$', fontsize=XBIG_SIZE)


    # Size of ticks
    axs[0].tick_params(axis='both', which='major', length=5, width=1)
    axs[1].tick_params(axis='both', which='major', length=5, width=1)
    axs[2].tick_params(axis='both', which='major', length=5, width=1)

    # Hide y ticks in 2nd and 3rd panes
    axs[1].axes.yaxis.set_visible(False)
    axs[2].axes.yaxis.set_visible(False)

    # axs[0].yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    # axs[0].ticker.LogFormatter(base=10.0)
    # import matplotlib.ticker as tick
    # axs[0].yaxis.set_major_formatter(tick.LogFormatter())

    # legend
    axs[0].legend(loc='upper right', frameon=False)


    saveFig('fiducial_profiles_RS_hist.pdf', fig, bbox_inches='tight')


make_histograms()

In [ ]:
# Makes Figure 2

u = 100 # 100 of 256 pixels (140 Mpc of 256/h Mpc)
p = 40 * resolution # zoom in on 40 pixels for profile comparison

def compare_4_profile_fields(STH2, NFW, FIRE, PRE, resolution, z):
    """Creates a plot comparing the fields from 4 CGM profiles (see variable names)."""

    vmin = 0
    #vmax = max(np.max(STH2), np.max(NFW), np.max(FIRE), np.max(PRE))
    #vhigh = vmax*0.1
    #vmax = int(round(vhigh, -int(math.floor(math.log10(vhigh)))))  
    vmax = 3000

    fig, axs = fields_comparison_plot([('NFW', NFW), ('FIRE', FIRE), ('Precipitation', PRE), ('Tophat $2R_{vir}$', STH2)], vmin, vmax)

    filename = 'stacked_field_images_{}_z{:.1f}.pdf'.format(resolution, z)
    saveFig(filename, fig, bbox_inches='tight')

f01=compare_4_profile_fields(STH_2_DM_to_0_1[0:p,0:p], NFW_DM_to_0_1[0:p,0:p], fire_DM_to_0_1[0:p,0:p], P_DM_to_0_1[0:p,0:p], resolution, 0.1)
f05=compare_4_profile_fields(STH_2_DM_to_0_5[0:p,0:p], NFW_DM_to_0_5[0:p,0:p], fire_DM_to_0_5[0:p,0:p], P_DM_to_0_5[0:p,0:p], resolution, 0.5)
f1=compare_4_profile_fields(STH_2_DM_to_1[0:p,0:p], NFW_DM_to_1[0:p,0:p], fire_DM_to_1[0:p,0:p], P_DM_to_1[0:p,0:p], resolution, 1.0)


In [ ]:
# Make an algorithm plot for STH using stacked field out to z=1

def algorithm_stages_plot(raw_field, removed_field, addition_field, final_field, orig_grid_size, vmax, zoom_rect: bool):
    vmin = 0
    fig, axs = fields_comparison_plot( [('Raw density field', raw_field), ('Halos removed', removed_field), ('Halo-associated gas', addition_field), ('Final halo density field', final_field)], vmin, vmax)
    
    if zoom_rect:
        rect = Rectangle((0,0),p,p,linewidth=1,edgecolor='r',facecolor="none")
        axs[3].add_patch(rect)
    
    extra = ''
    if zoom_rect:
        extra = '_wbox'
    filename = 'stacked_implot_add_remove_halos_{}{}.pdf'.format(orig_grid_size, extra)
    saveFig(filename, fig, bbox_inches='tight')
    
original_field = sum(configs[0].get_stacked_orig_field())
removed_field = sum(configs[0].get_stacked_removed_field())
added_field = sum(configs[0].get_stacked_addition_field())
final_field = STH_DM_to_1
    
algorithm_stages_plot(original_field[0:u,0:u], removed_field[0:u,0:u], added_field[0:4*u*resolution,0:4*u*resolution], final_field[0:4*u*resolution,0:4*u*resolution], 256, 2000, True)


In [ ]:
# Not for paper: Final density at various redshifts (NOT out to that redshift, just 1 box at that redshift)

vmin = 0
vmax = 500
x = 40 * 8
fig, axs = fields_comparison_plot([
    ('Precipitation z={:.2f}'.format(redshifts[0]), configs[4].get_final_field()[0][0:x,0:x]), 
     ('Precipitation z={:.2f}'.format(redshifts[3]), configs[4].get_final_field()[3][0:x,0:x]), 
     ('Precipitation z={:.2f}'.format(redshifts[5]), configs[4].get_final_field()[5][0:x,0:x]), 
     ('Precipitation z={:.2f}'.format(redshifts[8]), configs[4].get_final_field()[8][0:x,0:x])
], vmin, vmax)
saveFig("precipitation_z_boxes", fig, bbox_inches='tight')
fig, axs = fields_comparison_plot([
    ('Fire z={:.2f}'.format(redshifts[0]), configs[3].get_final_field()[0][0:x,0:x]), 
     ('Fire z={:.2f}'.format(redshifts[3]), configs[3].get_final_field()[3][0:x,0:x]), 
     ('Fire z={:.2f}'.format(redshifts[5]), configs[3].get_final_field()[5][0:x,0:x]), 
     ('Fire z={:.2f}'.format(redshifts[8]), configs[3].get_final_field()[8][0:x,0:x])
], vmin, vmax)
saveFig("fire_z_boxes", fig, bbox_inches='tight')

print("Sanity check on mean density of fields for Preciptation")
print(np.mean(configs[4].get_final_field()[0]))
print(np.mean(configs[4].get_final_field()[3]))
print(np.mean(configs[4].get_final_field()[5]))
print(np.mean(configs[4].get_final_field()[8]))

print("Sanity check on mean density of fields for Fire")
print(np.mean(configs[3].get_final_field()[0]))
print(np.mean(configs[3].get_final_field()[3]))
print(np.mean(configs[3].get_final_field()[5]))
print(np.mean(configs[3].get_final_field()[8]))
